In [7]:
print("Hello World!")

Hello World!


In [1]:
# Import necessary packages
import numpy as np
import scipy
from scipy.optimize import approx_fprime
from scipy import integrate
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

# Define some constants for examples
au2m       = 1.49598e11 # AU in meters
days2sec    = 86400. # Days in seconds
years2sec    = 31557600. # Years in seconds
mSun_kg     = 1.98892e30 # Mass of Sun in Kg
mEarth_kg   = 5.9742e24 # Mass of Earth in Kg
mJupiter_kg = 1.8987e27 # Mass of Jupiter in Kg
rSun_m = 6.9634e8 # Radius of Sun in meters
rJupiter_m = 6.9911e7 # Radius of Jupiter in meters
rEarth_m = 6.371e6 # Radius of Earth in meters
pMass = 1.6723e-27 # Mass of Proton in Kg
eMass = 9.1094e-31 # Mass of Electron in Kg
eCharge = 1.6e-19 # Elementary Charge in Coulombs
hDist = 5.29e-11 # Distance of Electron to nucleus in Hydrogen in meters
eVel = 2.18e6 # Velocity of electron around hydrogen nucleus in m/s
Earth_vel = 29784.8 # Velocity of Earth around Sun in m/s
G = 6.67e-11 # Gravitational Constant
K = 9.0e+9 # Coulomb Constant

In [ ]:
class transit:
    def __init__(self):
        self.placeholder = 1

class trajectory:
    def __init__(self):
        self.placeholder = 1

# Solve Kepler's equation for Eccentric Anomaly E
def kepler(E, M, e):
    return E - e * np.sin(E) - M
    
def params_to_traj(period, t0, e, m1, m2):
    t = np.linspace(0,period*days2sec,100)
    n = 2 * np.pi / (period*days2sec)
    M = n * t - t0*days2sec
    a = np.power(G*(m1 + m2)*mSun_kg/(4*np.pi**2)*(period*days2sec)**2, 1/3)

    E = np.array([fsolve(kepler, M_i, args=(M_i, e))[0] for M_i in M])

    x = a*(np.cos(E)-e)
    y = a*np.sqrt(1 - e**2)*np.sin(E)
    traj = np.transpose(np.array([np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x)), x/au2m, y/au2m, np.zeros(len(x))]))
    
    return traj, t/days2sec

def nBody_animation(num_particles, traj, t, radii):
    # Given the array-like input variable t, initialize variables that store
    # the number of time steps and the length of each step.
    n_timesteps = len(t)
    total_time = int(max(t))
    dt = round((total_time/n_timesteps), 2)
    # Initialize array of particle position data - random for now
    positions = np.random.randn(n_timesteps, num_particles, 3)
    # Iterate through array traj to replace values in positions array with those in traj
    for n in range(0,n_timesteps):
        for i in range(0,num_particles):
            for c in range(0,3):
                positions[n,i,c] = traj[n,3*i+c]
    # Create x, y, and z ranges based on the min, max values of x, y, and z
    # That appear in the positions array
    x_range = np.array([np.min(positions[:,:,0]), np.max(positions[:,:,0])])
    y_range = np.array([np.min(positions[:,:,1]), np.max(positions[:,:,1])])
    z_range = np.array([np.min(positions[:,:,2]), np.max(positions[:,:,2])])
    # Initialize the figure
    layout = go.Layout(width=600, height=600, title="computed N-body")
    fig = go.Figure(layout=layout)
    # Assign unique random colors to each particle
    # And add the initial plot (time step zero)
    particle_colors = {}
    for iparticle in range(num_particles):
        rgb = np.random.randint(0, 255, 3)
        color = f'rgb({[255, 255, 0]},{[0, 0, 139]}, {[0,0,0]})'
        particle_colors[iparticle] = color
        # Add initial scatter plot (marker)
        fig.add_trace(go.Scatter3d(
            x=[], y=[], z=[],
            mode='markers',
            marker=dict(size=5, color=color),
            name=f'Particle {iparticle}'
        ))
        # Add trajectory line
        fig.add_trace(go.Scatter3d(
            x=[], y=[], z=[],
            mode='lines',
            line=dict(color=color, width=2),
            name=f'Trajectory {iparticle}'
        ))
    # Assign length of trajectory tail
    tail_length = 1000
    # Create frames for each time step
    frames = []
    for k in range(n_timesteps):
        frame_data = []
        for iparticle in range(num_particles):
            # Current positions of particles (scatter plot)
            frame_data.append(go.Scatter3d(
                x=np.array(positions[k, iparticle, 0]),
                y=np.array(positions[k, iparticle, 1]),
                z=np.array(positions[k, iparticle, 2]),
                mode='markers',
                marker=dict(size=radii[iparticle], color=particle_colors[iparticle]),
                name=f'Particle {iparticle + 1}'
            ))
            # Particle trajectory tails (lines)
            start_index = max(0, k - tail_length + 1)  # Ensure we don’t try to make a trajectory start before 0
            frame_data.append(go.Scatter3d(
                x=positions[start_index:k+1, iparticle, 0],
                y=positions[start_index:k+1, iparticle, 1],
                z=positions[start_index:k+1, iparticle, 2],
                mode='lines',
                line=dict(color=particle_colors[iparticle], width=2),
                name=f'Trajectory {iparticle + 1}'
            ))
        frames.append(go.Frame(data=frame_data, name=f'frame{k}'))
    fig.frames = frames
    # Add animation controls
    fig.update_layout(
        updatemenus=[dict(
            type="buttons",
            buttons=[
                dict(label="Play",
                     method="animate",
                     args=[None, {"frame": {"duration": 50, "redraw": True},
                                  "fromcurrent": True, "transition": {"duration": 0}}]),
                dict(label="Pause",
                     method="animate",
                     args=[[None], {"frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate",
                                    "transition": {"duration": 0}}])
            ],
            direction="left",
            pad={"r": 10, "t": 87},
            showactive=False,
            x=0.1, xanchor="right", y=0, yanchor="top"
        )],
        sliders=[dict(
            active=0,
            yanchor="top", xanchor="left",
            currentvalue={"prefix": "Time: ", "suffix": " s"},
            pad={"b": 10, "t": 50}, len=0.9, x=0.1, y=0,
            steps=[{"method": "animate",
                    "args": [[f'frame{k}'],
                             {"frame": {"duration": 10, "redraw": True},
                              "mode": "immediate",
                              "transition": {"duration": 0}}], "label": f"{k * dt:.2f}"}
                   for k in range(n_timesteps)]
        )]
    )
    # Update initial layout to include the custom axis ranges set above
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='x (m)', range=[x_range[0], x_range[1]], autorange=False),
            yaxis=dict(title='y (m)', range=[y_range[0], y_range[1]], autorange=False),
            zaxis=dict(title='z (m)', range=[z_range[0], z_range[1]], autorange=False),
            aspectratio=dict(x=1, y=1, z=1)
        )
    )
    return fig

In [13]:
traj, t = params_to_traj(5.96727, -1551.281773, 0.000479, 1.183338, 0.201989)
print(np.shape(traj))
print(t)
fig_nbody_animation = nBody_animation(2, traj, t, [10*1.16, 10*0.2])
fig_nbody_animation.show()

(100, 6)
[0.         0.06027545 0.12055091 0.18082636 0.24110182 0.30137727
 0.36165273 0.42192818 0.48220364 0.54247909 0.60275455 0.66303
 0.72330545 0.78358091 0.84385636 0.90413182 0.96440727 1.02468273
 1.08495818 1.14523364 1.20550909 1.26578455 1.32606    1.38633545
 1.44661091 1.50688636 1.56716182 1.62743727 1.68771273 1.74798818
 1.80826364 1.86853909 1.92881455 1.98909    2.04936545 2.10964091
 2.16991636 2.23019182 2.29046727 2.35074273 2.41101818 2.47129364
 2.53156909 2.59184455 2.65212    2.71239545 2.77267091 2.83294636
 2.89322182 2.95349727 3.01377273 3.07404818 3.13432364 3.19459909
 3.25487455 3.31515    3.37542545 3.43570091 3.49597636 3.55625182
 3.61652727 3.67680273 3.73707818 3.79735364 3.85762909 3.91790455
 3.97818    4.03845545 4.09873091 4.15900636 4.21928182 4.27955727
 4.33983273 4.40010818 4.46038364 4.52065909 4.58093455 4.64121
 4.70148545 4.76176091 4.82203636 4.88231182 4.94258727 5.00286273
 5.06313818 5.12341364 5.18368909 5.24396455 5.30424    5.3

ValueError: 
    Invalid value of type 'builtins.str' received for the 'color' property of scatter3d.marker
        Received value: 'rgb([255, 255, 0],[0, 0, 139], [0, 0, 0])'

    The 'color' property is a color and may be specified as:
      - A hex string (e.g. '#ff0000')
      - An rgb/rgba string (e.g. 'rgb(255,0,0)')
      - An hsl/hsla string (e.g. 'hsl(0,100%,50%)')
      - An hsv/hsva string (e.g. 'hsv(0,100%,100%)')
      - A named CSS color:
            aliceblue, antiquewhite, aqua, aquamarine, azure,
            beige, bisque, black, blanchedalmond, blue,
            blueviolet, brown, burlywood, cadetblue,
            chartreuse, chocolate, coral, cornflowerblue,
            cornsilk, crimson, cyan, darkblue, darkcyan,
            darkgoldenrod, darkgray, darkgrey, darkgreen,
            darkkhaki, darkmagenta, darkolivegreen, darkorange,
            darkorchid, darkred, darksalmon, darkseagreen,
            darkslateblue, darkslategray, darkslategrey,
            darktur